<a href="https://colab.research.google.com/github/abhi91420/Resume_Automation/blob/main/Resume_Builder_Automate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Resume Builder Automation

- Input : Job description
- Output : Resume with relavent skills to the job

### Creating a document with the information
---
Required package : Python Docx 

- install command : ```!pip install python-docx```


In [49]:
!pip install python-docx

### Imports


In [50]:
import os
import pandas as pd
import docx
from os import path
import pickle
from docx.enum.text import *
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
# creating directory in google drive 

os.chdir("/content/drive/MyDrive")
try:
  os.chdir("/content/drive/MyDrive/Generated Resumes/data")
except FileNotFoundError:
  os.mkdir("Generated Resumes")
  os.chdir("/content/drive/MyDrive/Generated Resumes")
  os.mkdir("data")
  os.chdir("/content/drive/MyDrive/Generated Resumes/data")

### Custom Functions

In [52]:
# custom function 

def heading(x,n):
  return resume.add_heading(x,n)

def para(x):
  return  resume.add_paragraph(x)

def center(x):
  x.alignment = WD_ALIGN_PARAGRAPH.CENTER
  
def project(txt,name,abstract,contrib):
  para("")
  heading(txt.upper()+" - "+name,4)
  para("Abstract -"+abstract)
  heading("Contribution:",4)
  para("")
  para(contrib)

### Input data preprocessing

we paste the job description and we extract meaningful data from it


In [53]:
def job_description(x):
  
  job_desc = x
  esc_char = [' ',',',':','/','\n','\t','-','$',"\\",'.']
  for ch in esc_char:
    job_desc = job_desc.replace(ch,'')
  
  job_desc_norm = job_desc.lower()
  #tech_set = pd.read_csv("https://raw.githubusercontent.com/abhi91420/Resume_Automation/main/data/list_of_softwares.csv",header = None,error_bad_lines= False)
  tech_set = pd.read_csv("https://raw.githubusercontent.com/abhi91420/Resume_Automation/main/data/list_ece.csv",header = None)
  tech_set = tech_set[0]
  tech_set = tech_set.drop_duplicates()
  
  try:
    for i in range(tech_set.size):
      tech_set_list.append(tech_set[i])
    
  
  except KeyError:
    pass
 
  return job_desc_norm,tech_set_list

### Normalize the data

In [54]:
def Normalize(x):
  temp = []
  char = [' ',',',':','/','\n','\t','-','$',"\\",'.']
  for i in range(len(x)):
    x[i] = str(x[i].lower())
    for ch in char:
      x[i] = x[i].replace(ch,'')
    temp.append(x[i])
  return temp

### Sections


In [55]:
def search(job_desc_norm,tech_set_list):
  
  tech_set_norm = Normalize(tech_set_list)
  for i in tech_set_norm:
    if i in job_desc_norm:
      found_set.append(i)
  if len(found_set)%2 != 2:
    found_set.append("")

  print("=========================================")
  print("Found Skills Required :")
  print("=========================================")
  
  for item in found_set:
    print(item)
  
  
  

In [56]:
# add objective/summary

def objective():
  heading("Objective Summary",2)
  if (path.exists("objective.p")):
    print("Objective Summary Found")
    objective = pickle.load(open("objective.p","rb"))
  else:
    objective = input("Enter Summary : ")
    pickle.dump(objective,open("objective.p","wb"))
  para("")
  para(objective)


In [57]:
# add work experience

def work_exp():
  heading("Work Experience",2)
  para("")
  if (path.exists("work_exp.p")):
    print("Work Experience Found")
    temp = pickle.load(open("work_exp.p","rb"))
  else:
    work_title = input("Enter Your Work title : ")
    company = input("Enter Company's Name : ")
    heading(work_title  + "-" + company,2)
    heading("Responsibilities and duties :",3)
    duties = []
    print("\nEnter Duties : (enter q to stop entering duties) ")
    flag = ""
    while flag != "q":
      flag = input("enter a duty : ")
      duties.append(". "+ flag)
      temp = ""
    for duty in duties[:-1]:
      temp = temp+duty+"\n"
    pickle.dump(temp,open("work_exp.p","wb"))
  para(temp)


In [58]:
# add education

def education():
  if path.exists("education_data.p"):
    print("Education Found")
    temp = pickle.load(open("education_data.p","rb"))
  else:
    print("\nEducation Section \n")
    education = input("Enter Your Name of the Degree :")
    institute = input("Name of the institute : ")
    institute_location = input("City : ") +","+input("State :")
    years = input("From year : ")+" - "+input("To year : ")
    heading("Education",2)
    para("")
    heading(education,3)
    temp = institute+","+institute_location+"\n"+years
    pickle.dump(temp,open("education_data.p","wb"))
  para(temp)

In [59]:
# add skills

def skills():
  heading("Skills",2)
  para("")
  skill_table = resume.add_table(rows = 0,cols = 2)
  count = 0
  for i in range(int(len(found_set)/2)):   
      table_cells = skill_table.add_row().cells    
      table_cells[0].text = found_set[count]
      count += 1
      table_cells[1].text = found_set[count]
      count += 1

In [60]:
# add projects

def projects():
  heading("Projects ",2)
  flag = True
  if path.exists("no_of_projects.p"):
    print("Projects Found")
    no_of_projects = pickle.load(open("no_of_projects.p","rb"))

    # impliment multiple loads for no of projects
    
    for i in range(int(no_of_projects )):
      if path.exists("project"+str(i)+"_info.d"):
        temp = pickle.load(open("project"+str(i)+"_info.d","rb"))
        project(temp[0],temp[1],temp[2],temp[3])
        
  else:
   
    no_of_projects = int(input("Enter no of projects : "))  
    pickle.dump(no_of_projects,open("no_of_projects.p","wb"))
    for i in range(no_of_projects):
      txt = input("Enter Technology used : ")
      name = input("Enter Title of the project : ")
      abstract = input("Enter abstract : ")
      contribution = input("Enter Contribution : ")
      temp = [txt,name,abstract,contribution]
      pickle.dump(temp,open("project"+str(i)+"_info.d","wb"))
      project(temp[0],temp[1],temp[2],temp[3])
 

### Required data for document entry

- user enters his personal data (Name, No, address etc)
- prewritten set of objective paragraphs

In [61]:
# initialize document object

resume = docx.Document()

In [62]:
# User prompt information

if (path.exists("user_info.p")):
  user_info = pickle.load(open("user_info.p","rb"))
  name = user_info["name"]
  line_2 = user_info["address"]+','+user_info["phone_no"]+','+user_info["email_id"]
else:
   name = input("Enter Your Name : ")
   address = input("Enter your Address : ")
   phone_no = input("Enter Your Contact No :")
   email_id = input("Enter Email id : ")
   line_2 = address+','+phone_no+','+email_id
   user_info = {"name":name,"address":address,"phone_no":phone_no,"email_id":email_id}
   pickle.dump(user_info,open("user_info.p","wb"))

In [63]:
# add the user prompt info to the docxfile 

title_1 = heading(name,1)
center(title_1)
title_2 = heading(line_2,2)
center(title_2)
# Line space 
para("")

In [48]:
# Save the document

if __name__ == "__main__":
  print("===========================================")
  print("Resume Creation Started")
  print("===========================================")
  job_desc= input("Copy Paste Job Description : ")
  found_set = []
  flag = True
  tech_set_list= []
  a,b = job_description(job_desc)
  search(a,b)
  try:
    function_caller_dict= {1:"Work experience",2:"Education",3:"Skills",4:"Projects"}
    objective()
    template_selection = int(input("\n1. Default Sections\n2. Custom Sections\nEnter your choice no :"))
     
    if template_selection == 1:
      work_exp()
      education()
      skills()
      projects()
    elif template_selection ==2:
      print("Select a section to fill\n")
      for i in function_caller_dict.keys():
        print(i,".",function_caller_dict[i])
        temp = 0
        while temp < 4:
          select = input("Enter a section no : ")
          if select == 1:
            work_exp()
            temp += 1
          elif select == 2:
            education()
            temp += 1
          elif select == 3:
            skills()
            temp += 1
          elif select == 4:
            for i in range(input("No of projects : ")):
              project()
            temp += 1
          else:
            print("invalid entry")
      else:
        print("Invalid")
    role = str(input("Title of the job your applying for : ")).capitalize()
    company = str(input("Name of the company your applying for :")).capitalize()
    os.chdir("/content/drive/MyDrive/Generated Resumes")
    resume.save(company+" "+role+" Resume"+".docx")
  except KeyboardInterrupt:
      print("Exited")
  print("Resume Saved in your drive in Generated Resumes")

Resume Creation Started
Copy Paste Job Description : Engineering Designs	 AutoCAD	 Design Verification	 Electronic Trouble shooting	 Power Simulator	 Oscilloscopes and Generators	 Test Engineering	 Microcontroller Softwares
Found Skills Required :
engineeringdesigns
autocad
designverification
electronictroubleshooting
powersimulator
oscilloscopesandgenerators
testengineering
microcontrollersoftwares

Objective Summary Found
Exited
